In [2]:
# import findspark

# findspark.init()
sc # STARTING SPARK APPLICATION


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('UPGRAD_ETL').master("local").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

#creating schema using "StructType"

Schema = StructType([StructField('year', IntegerType(), nullable = True),
                        StructField('month', StringType(), True),
                        StructField('day', IntegerType(), True),
                        StructField('weekday', StringType(), True),
                        StructField('hour', IntegerType(), True),
                        StructField('atm_status', StringType(), True),
                        StructField('atm_id', StringType(), True),
                        StructField('atm_manufacturer', StringType(), True),
                        StructField('atm_location', StringType(), True),
                        StructField('atm_streetname', StringType(), True),
                        StructField('atm_street_number', IntegerType(), True),
                        StructField('atm_zipcode', IntegerType(), True),
                        StructField('atm_lat', DoubleType(), True),
                        StructField('atm_lon', DoubleType(), True),
                        StructField('currency', StringType(), True),
                        StructField('card_type', StringType(), True),
                        StructField('transaction_amount', IntegerType(), True),
                        StructField('service', StringType(), True),
                        StructField('message_code', StringType(), True),
                        StructField('message_text', StringType(), True),
                        StructField('weather_lat', DoubleType(), True),
                        StructField('weather_lon', DoubleType(), True),
                        StructField('weather_city_id', IntegerType(), True),
                        StructField('weather_city_name', StringType(), True),
                        StructField('temp', DoubleType(), True),
                        StructField('pressure', IntegerType(), True),
                        StructField('humidity', IntegerType(), True),
                        StructField('wind_speed', IntegerType(), True),
                        StructField('wind_deg', IntegerType(), True),
                        StructField('rain_3h', DoubleType(), True),
                        StructField('clouds_all', IntegerType(), True),
                        StructField('weather_id', IntegerType(), True),
                        StructField('weather_main', StringType(), True),
                        StructField('weather_description', StringType(), True)])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.format("csv")\
                .option("header", "false")\
                .option("inferSchema", "false")\
                .schema(Schema)\
                .option("mode", "FAILFAST")\
                .load("/user/hadoop/UPGRAD/ETL/part-m-00000")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.printSchema() # vewing the Schema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

In [7]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

## GENERATING THE DIMENSION AND FACT TABLES

### Establishing a dataframe for the Location Dimension

In [8]:
# creating a temporary df and selecting required columns & making sure records are distinct 
location = df.select("atm_location", "atm_streetname", "atm_street_number", "atm_zipcode", "atm_lat", "atm_lon").distinct()
location.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

# creating the primary key column
dim_location = location.select(row_number().over(Window.orderBy(location[0])).alias("location_id"), "*")
dim_location.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
|location_id|        atm_location|atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|  ÃƒËœstergade|                6|       9440| 57.162|   9.73|
|          2|      Aalborg Hallen|  Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|      Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|      Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|      Hobrovej|              440|       9200| 57.005|  9.881|
+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
only showing top 5 rows

In [10]:
# renaming the columns as per requirement
DIM_LOCATION = dim_location.withColumnRenamed('atm_location','location')\
                            .withColumnRenamed('atm_streetname','streetname')\
                            .withColumnRenamed('atm_street_number','street_number')\
                            .withColumnRenamed('atm_zipcode','zipcode')\
                            .withColumnRenamed('atm_lat','lat')\
                            .withColumnRenamed('atm_lon','lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# checking that all required columns are present and named correctly
DIM_LOCATION.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon']

In [12]:
# validating the count of the dataframe
DIM_LOCATION.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

### Establishing a dataframe for the Card Type Dimension

In [13]:
card_type =  df.select('card_type').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# creating the primary key column
DIM_CARD_TYPE = card_type.select(row_number().over(Window.orderBy(card_type[0])).alias("card_type_id"), "*")
DIM_CARD_TYPE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------------+
|card_type_id|          card_type|
+------------+-------------------+
|           1|             CIRRUS|
|           2|            Dankort|
|           3|    Dankort - on-us|
|           4|        HÃƒÂ¦vekort|
|           5|HÃƒÂ¦vekort - on-us|
+------------+-------------------+
only showing top 5 rows

In [15]:
# checking that all required columns are present and named correctly
DIM_CARD_TYPE.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type_id', 'card_type']

In [16]:
# validating the count of the dataframe
DIM_CARD_TYPE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

### Establishing a dataframe for the  ATM Dimension

In [17]:
# creating a temporary df and selecting required columns
atm = df.select(col('atm_id').alias('atm_number'), 'atm_manufacturer', 'atm_lat', 'atm_lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
atm = atm.join(dim_location, on = ['atm_lat', 'atm_lon'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# checking to columns in the joined df
atm.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_lat', 'atm_lon', 'atm_number', 'atm_manufacturer', 'location_id', 'atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode']

In [20]:
# selecting the required columns and making sure records are distinct
dim_atm = atm.select('atm_number', 'atm_manufacturer', 'location_id').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
DIM_ATM = dim_atm.select(row_number().over(Window.orderBy(dim_atm[0])).alias('atm_id'), 'atm_number', 'atm_manufacturer', col('location_id').alias('atm_location_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# checking that all required columns are present and named correctly
DIM_ATM.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_id', 'atm_number', 'atm_manufacturer', 'atm_location_id']

In [23]:
# validating the count of the dataframe
DIM_ATM.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156

In [24]:
DIM_ATM.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+---------------+
|atm_id|atm_number|atm_manufacturer|atm_location_id|
+------+----------+----------------+---------------+
|     1|         1|             NCR|             74|
|     2|        10|             NCR|             76|
|     3|       100|             NCR|             56|
|     4|       100|             NCR|             86|
|     5|       100|             NCR|             87|
|     6|       101|             NCR|             17|
|     7|       102|             NCR|              3|
|     8|       102|             NCR|             91|
|     9|       102|             NCR|              4|
|    10|       103| Diebold Nixdorf|            103|
|    11|       104|             NCR|             58|
|    12|       104|             NCR|            108|
|    13|       104|             NCR|            109|
|    14|       105| Diebold Nixdorf|             76|
|    15|       106|             NCR|             79|
|    16|       106|             NCR|          

### Establishing a dataframe for the Date Dimension

In [25]:
# creating a temporary df and selecting required columns
date = df.select('year', 'month', 'day', 'hour', 'weekday')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
date = date.withColumn('full_date_time', to_timestamp(concat(date.year, lit('-'), date.month, lit('-'),date.day, lit(' '), date.hour), 'yyyy-MMMM-d H'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
date.show(5, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+-------+-------------------+
|year|month  |day|hour|weekday|full_date_time     |
+----+-------+---+----+-------+-------------------+
|2017|January|1  |0   |Sunday |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-01-01 00:00:00|
+----+-------+---+----+-------+-------------------+
only showing top 5 rows

In [28]:
# selecting the required columns and making sure records are distinct
date = date.select('full_date_time', 'year', 'month', 'day', 'hour', 'weekday').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
DIM_DATE = date.select(row_number().over(Window.orderBy(date[0])).alias('date_id'), '*')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# checking that all required columns are present and named correctly
DIM_DATE.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['date_id', 'full_date_time', 'year', 'month', 'day', 'hour', 'weekday']

In [31]:
# validating the count of the dataframe
DIM_DATE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

### Establishing a dataframe for theTransaction Fact Table

In [32]:
fact_loc = df.withColumnRenamed('atm_location','location')\
    .withColumnRenamed('atm_streetname','streetname')\
    .withColumnRenamed('atm_street_number','street_number')\
    .withColumnRenamed('atm_zipcode','zipcode')\
    .withColumnRenamed('atm_lat','lat')\
    .withColumnRenamed('atm_lon','lon')

# joining original dataframe with DIM_LOCTION
fact_loc = fact_loc.join(DIM_LOCATION, on = ['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
fact_loc = fact_loc.withColumnRenamed('atm_id', 'atm_number').withColumnRenamed('location_id', 'atm_location_id')

# joining the dataframe with DIM_ATM
fact_atm = fact_loc.join(DIM_ATM, on = ['atm_number', 'atm_manufacturer', 'atm_location_id'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# performing necessary transformations, same as done to atm table
fact_atm = fact_atm.withColumnRenamed('atm_location_id', 'weather_loc_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# joining the dataframe with DIM_DATE
fact_date = fact_atm.join(DIM_DATE, on = ['year', 'month', 'day', 'hour', 'weekday'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# joining the dataframe with DIM_CARD_TYPE
fact_atm_trans = fact_date.join(DIM_CARD_TYPE, on = ['card_type'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# creating primary key of fact table
FACT_ATM_TRANS = fact_atm_trans.withColumn("trans_id", row_number().over(Window.orderBy('date_id')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# viewing the list of columns
FACT_ATM_TRANS.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id', 'trans_id']

In [39]:
# selecting and arranging only the required columns according to the target model
FACT_ATM_TRANS = FACT_ATM_TRANS.select('trans_id', 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 
'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 
'clouds_all', 'weather_id', 'weather_main', 'weather_description')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# checking that all required columns are present and named correctly
FACT_ATM_TRANS.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['trans_id', 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']

In [41]:
# validating the count of the dataframe
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [42]:
FACT_ATM_TRANS.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|       1|    38|            62|      1|           7|    Active|     DKK|Withdrawal|              7951|        null|        null|   0.25|        92|       500|        Rain|         light rain|
+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
only showing top 1 row

## Writing the PySpark Dataframes to tables to S3 bucket

In [43]:
# writing data from pyspark df 'dim_location' in csv format to dim_location folder in S3 bucket 'etlprojectbysimran'
DIM_LOCATION.coalesce(1).write.format('csv').option('header','false').save('s3://s3bucket4upgradredshift/ETL-Upgrad/DIM_LOCATION/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# writing data from pyspark df 'dim_atm' in csv format to dim_atm folder in S3 bucket 'etlprojectbysimran'
DIM_ATM.coalesce(1).write.format('csv').option('header','false').save('s3://s3bucket4upgradredshift/ETL-Upgrad/DIM_ATM/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# writing data from pyspark df 'dim_data' in csv format to dim_data folder in S3 bucket 'etlprojectbysimran'
DIM_DATE.coalesce(1).write.format('csv').option('header','false').save('s3://s3bucket4upgradredshift/ETL-Upgrad/DIM_DATE/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# writing data from pyspark df 'dim_card_type' in csv format to dim_card_type folder in S3 bucket 'etlprojectbysimran'
DIM_CARD_TYPE.coalesce(1).write.format('csv').option('header','false').save('s3://s3bucket4upgradredshift/ETL-Upgrad/DIM_CARD/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
# writing data from pyspark df 'fact_atm_trans' in csv format to fact_atm_trans folder in S3 bucket 'etlprojectbysimran'
FACT_ATM_TRANS.coalesce(1).write.format('csv').option('header','false').save('s3://s3bucket4upgradredshift/ETL-Upgrad/FACT_ATM_TRANS/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…